In [42]:
import pandas as pd
import numpy as np
from pandas.core.arrays import integer
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# pip install openpyxl - to read the xlsx

In [ ]:
ptl_df = pd.read_excel('./PTL.xlsx', decimal=',')
ptl_df

In [ ]:
ptl_df.columns

In [ ]:
# Lista de columnas a convertir a float
columnas_a_convertir_float = ['Hb 1', 'Hb 2', 'Hb 3', 'Hb 4',
       'Hb 5', 'RBC 1', 'RBC 2', 'RBC 3', 'RBC 4', 'RBC 5', 'HTC 1', 'HTC 2',
       'HTC 3', 'HTC 4', 'HTC 5', 'VCM 1', 'VCM 2', 'VCM 3', 'VCM 4', 'VCM 5',
       'HCM 1', 'HCM 2', 'HCM 3', 'HCM 4', 'HCM 5', 'CHCM 1', 'CHCM 2',
       'CHCM 3', 'CHCM 4', 'CHCM 5', 'ADE 1', 'ADE 2', 'ADE 3', 'ADE 4',
       'ADE 5', 'PLT 1', 'PLT 2', 'PLT 3', 'PLT 4', 'PLT 5']

# Convierte las columnas a tipo float
ptl_df[columnas_a_convertir_float] = ptl_df[columnas_a_convertir_float].astype(float)

ptl_df.dtypes

In [ ]:
ptl_df.describe()

In [ ]:
ptl_df[['Hb 1', 'Hb 2', 'Hb 3', 'Hb 4', 'Hb 5']].describe()

In [ ]:
ptl_df[['PLT 1', 'PLT 2', 'PLT 3', 'PLT 4', 'PLT 5']].describe()

EL MODELO

In [ ]:
# Define las categorías de riesgo
ptl_df["Categoría de Riesgo"] = ""

# Calcula el coeficiente de variación (CV) entre las columnas 'Hb 1' a 'Hb 5' y almacénalo en una nueva columna
ptl_df["CV_Hb"] = ptl_df[["Hb 1", "Hb 2", "Hb 3", "Hb 4", "Hb 5"]].std(axis=1) / ptl_df[
    ["Hb 1", "Hb 2", "Hb 3", "Hb 4", "Hb 5"]
].mean(axis=1)
ptl_df["CV_VCM"] = ptl_df[["VCM 1", "VCM 2", "VCM 3", "VCM 4", "VCM 5"]].std(
    axis=1
) / ptl_df[["VCM 1", "VCM 2", "VCM 3", "VCM 4", "VCM 5"]].mean(axis=1)


# Define las categorías de riesgo en función de CV_Hb y CV_VCM
# Ajusta los umbrales según tus criterios
cv_hb_threshold = 0.07
cv_vcm_threshold = 0.07

# Condiciones para clasificar a los pacientes
anemia_condition = (ptl_df["CV_Hb"] > cv_hb_threshold) & (
    ptl_df["CV_VCM"] > cv_vcm_threshold
)
no_anemia_condition = ~anemia_condition

# Asigna las categorías de riesgo
ptl_df.loc[anemia_condition, "Categoría de Riesgo"] = "Propenso a Desarrollar Anemia"
ptl_df.loc[no_anemia_condition, "Categoría de Riesgo"] = "Sin Riesgo de Anemia"

# Define las características clínicas (X) y las etiquetas (y) para el modelo de anemia
X_anemia = ptl_df[["CV_Hb", "CV_VCM"]]
y_anemia = ptl_df["Categoría de Riesgo"]

# Entrena un modelo de clasificación (por ejemplo, Random Forest) para predecir anemia
model_anemia = RandomForestClassifier(n_estimators=100, random_state=42)
model_anemia.fit(X_anemia, y_anemia)

# Realiza predicciones para todos los pacientes
predicciones_anemia = model_anemia.predict(X_anemia)

# Muestra el DataFrame con las predicciones de anemia
ptl_df.head(5)

In [ ]:
ptl_df[ptl_df['Categoría de Riesgo'] == 'Sin Riesgo de Anemia']

In [ ]:
ptl_df[ptl_df['Categoría de Riesgo'] == 'Propenso a Desarrollar Anemia']

Uso de nuevos datos

In [ ]:
nuevos_pacientes_df = pd.read_excel('nuevos_pacientes.xlsx')

nuevos_pacientes_df.head()

In [80]:
# Calcular el promedio de Hb para los nuevos datos
nuevos_pacientes_df["CV_Hb"] = nuevos_pacientes_df[
    ["Hb 1", "Hb 2", "Hb 3", "Hb 4", "Hb 5"]
].std(axis=1) / nuevos_pacientes_df[["Hb 1", "Hb 2", "Hb 3", "Hb 4", "Hb 5"]].mean(
    axis=1
)

np.random.seed(42)  # Para reproducibilidad

nuevos_pacientes_df["VCM 1"] = np.random.randint(80, 100, size=len(nuevos_pacientes_df))
nuevos_pacientes_df["VCM 2"] = np.random.randint(80, 100, size=len(nuevos_pacientes_df))
nuevos_pacientes_df["VCM 3"] = np.random.randint(80, 100, size=len(nuevos_pacientes_df))
nuevos_pacientes_df["VCM 4"] = np.random.randint(80, 100, size=len(nuevos_pacientes_df))
nuevos_pacientes_df["VCM 5"] = np.random.randint(80, 100, size=len(nuevos_pacientes_df))

nuevos_pacientes_df["CV_VCM"] = nuevos_pacientes_df[
    ["VCM 1", "VCM 2", "VCM 3", "VCM 4", "VCM 5"]
].std(axis=1) / nuevos_pacientes_df[["VCM 1", "VCM 2", "VCM 3", "VCM 4", "VCM 5"]].mean(
    axis=1
)


X_anemia_test = nuevos_pacientes_df[["CV_Hb", "CV_VCM"]]

# Realizar predicciones con el modelo entrenado
nuevas_predicciones = model_anemia.predict(X_anemia_test)

# Agregar las predicciones al DataFrame de nuevos pacientes
nuevos_pacientes_df["Categoría de Riesgo"] = nuevas_predicciones

In [ ]:
nuevos_pacientes_df[nuevos_pacientes_df['Categoría de Riesgo'] == 'Sin Riesgo de Anemia']

In [ ]:
nuevos_pacientes_df[nuevos_pacientes_df['Categoría de Riesgo'] == 'Propenso a Desarrollar Anemia']

In [ ]:
# Calcular el recuento de pacientes en cada categoría de riesgo
recuento_categorias = nuevos_pacientes_df['Categoría de Riesgo'].value_counts()

# Mostrar el recuento de pacientes en cada categoría de riesgo
print("Recuento de Pacientes por Categoría de Riesgo:")
print(recuento_categorias)